<a href="https://www.kaggle.com/code/tom99763/2x-yolo-flayer-meta-training-final-ver?scriptVersionId=267876190" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.metrics import roc_auc_score
from typing import List, Optional
from scipy.optimize import minimize
import pickle


import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
import lightgbm as lgb
import joblib
from lightgbm import early_stopping
import xgboost as xgb

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import LabelEncoder
from xgboost.callback import EarlyStopping

xgb.set_config(verbosity=0)

In [2]:
for name in ['lgb', 'xgb', 'cat']:
    if not os.path.exists(name):
        os.mkdir(name)

In [3]:
def weighted_multilabel_auc(
    y_true: np.ndarray,
    y_scores: np.ndarray,
    class_weights: Optional[List[float]] = None,
) -> float:
    """Compute weighted AUC for multilabel classification.

    Parameters:
    -----------
    y_true : np.ndarray of shape (n_samples, n_classes)
        True binary labels (0 or 1) for each class
    y_scores : np.ndarray of shape (n_samples, n_classes)
        Target scores (probability estimates or decision values)
    class_weights : array-like of shape (n_classes,), optional
        Weights for each class. If None, uniform weights are used.
        Weights will be normalized to sum to 1.

    Returns:
    --------
    weighted_auc : float
        The weighted average AUC

    Raises:
    -------
    ValueError
        If any class does not have both positive and negative samples
    """
    y_true = np.asarray(y_true)
    y_scores = np.asarray(y_scores)
    n_classes = y_true.shape[1]

    individual_aucs = roc_auc_score(y_true, y_scores, average=None)

    if class_weights is None:  # Uniform weights
        weights_array = np.ones(n_classes)
    else:
        weights_array = np.asarray(class_weights)

    if len(weights_array) != n_classes:
        raise ValueError(
            f'Number of weights ({len(weights_array)}) must match '
            f'number of classes ({n_classes})'
        )

    # Check for non-negative weights
    if np.any(weights_array < 0):
        raise ValueError('All class weights must be non-negative')

    # Check that at least one weight is positive
    if np.sum(weights_array) == 0:
        raise ValueError('At least one class weight must be positive')

    # Normalize weights to sum to 1
    weights_array = weights_array / np.sum(weights_array)

    # Compute weighted average
    return np.sum(individual_aucs * weights_array)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.metrics import roc_auc_score
from typing import List, Optional
from scipy.optimize import minimize
import pickle


import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
import lightgbm as lgb
import joblib
from lightgbm import early_stopping
import xgboost as xgb

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import LabelEncoder
from xgboost.callback import EarlyStopping

xgb.set_config(verbosity=0)
for name in ['lgb', 'xgb', 'cat']:
    if not os.path.exists(name):
        os.mkdir(name)
def weighted_multilabel_auc(
    y_true: np.ndarray,
    y_scores: np.ndarray,
    class_weights: Optional[List[float]] = None,
) -> float:
    """Compute weighted AUC for multilabel classification.

    Parameters:
    -----------
    y_true : np.ndarray of shape (n_samples, n_classes)
        True binary labels (0 or 1) for each class
    y_scores : np.ndarray of shape (n_samples, n_classes)
        Target scores (probability estimates or decision values)
    class_weights : array-like of shape (n_classes,), optional
        Weights for each class. If None, uniform weights are used.
        Weights will be normalized to sum to 1.

    Returns:
    --------
    weighted_auc : float
        The weighted average AUC

    Raises:
    -------
    ValueError
        If any class does not have both positive and negative samples
    """
    y_true = np.asarray(y_true)
    y_scores = np.asarray(y_scores)
    n_classes = y_true.shape[1]

    # Get AUC for each class
    individual_aucs = roc_auc_score(y_true, y_scores, average=None)

    # Handle weights
    if class_weights is None:  # Uniform weights
        weights_array = np.ones(n_classes)
    else:
        weights_array = np.asarray(class_weights)

    # Check weight dimensions
    if len(weights_array) != n_classes:
        raise ValueError(
            f'Number of weights ({len(weights_array)}) must match '
            f'number of classes ({n_classes})'
        )

    # Check for non-negative weights
    if np.any(weights_array < 0):
        raise ValueError('All class weights must be non-negative')

    # Check that at least one weight is positive
    if np.sum(weights_array) == 0:
        raise ValueError('At least one class weight must be positive')

    # Normalize weights to sum to 1
    weights_array = weights_array / np.sum(weights_array)

    # Compute weighted average
    return np.sum(individual_aucs * weights_array)


LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation'
]

# YOLO label mappings
YOLO_LABELS_TO_IDX = {
    'Anterior Communicating Artery': 0,
    'Basilar Tip': 1,
    'Left Anterior Cerebral Artery': 2,
    'Left Infraclinoid Internal Carotid Artery': 3,
    'Left Middle Cerebral Artery': 4,
    'Left Posterior Communicating Artery': 5,
    'Left Supraclinoid Internal Carotid Artery': 6,
    'Other Posterior Circulation': 7,
    'Right Anterior Cerebral Artery': 8,
    'Right Infraclinoid Internal Carotid Artery': 9,
    'Right Middle Cerebral Artery': 10,
    'Right Posterior Communicating Artery': 11,
    'Right Supraclinoid Internal Carotid Artery': 12
}

YOLO_INV_MAP = {v: k for k, v in YOLO_LABELS_TO_IDX.items()}

YOLO_LABELS = sorted(list(YOLO_LABELS_TO_IDX.keys()))

loc_cols = [f"loc_prob_{i}" for i in range(13)]

feature_cols = ['aneurysm_prob'] + loc_cols + LABEL_COLS +  ['Aneurysm Present'] + ['PatientAge', 'PatientSex']
yolo2flayer_loc_cols = [f'loc_prob_{LABEL_COLS.index(YOLO_INV_MAP[int(loc_col[-1])])}' for loc_col in loc_cols]

N = 14

In [5]:
list(enumerate(zip(YOLO_LABELS, yolo2flayer_loc_cols)))

[(0, ('Anterior Communicating Artery', 'loc_prob_6')),
 (1, ('Basilar Tip', 'loc_prob_11')),
 (2, ('Left Anterior Cerebral Artery', 'loc_prob_7')),
 (3, ('Left Infraclinoid Internal Carotid Artery', 'loc_prob_0')),
 (4, ('Left Middle Cerebral Artery', 'loc_prob_4')),
 (5, ('Left Posterior Communicating Artery', 'loc_prob_9')),
 (6, ('Left Supraclinoid Internal Carotid Artery', 'loc_prob_2')),
 (7, ('Other Posterior Circulation', 'loc_prob_12')),
 (8, ('Right Anterior Cerebral Artery', 'loc_prob_8')),
 (9, ('Right Infraclinoid Internal Carotid Artery', 'loc_prob_1')),
 (10, ('Right Middle Cerebral Artery', 'loc_prob_6')),
 (11, ('Right Posterior Communicating Artery', 'loc_prob_11')),
 (12, ('Right Supraclinoid Internal Carotid Artery', 'loc_prob_7'))]

In [6]:
df_main = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv')

In [7]:
# Load data
df_flayer = pd.read_csv('/kaggle/input/iad-model/oof_df_cv7722_seg_aux.csv')
df_meta = df_main.copy()[['SeriesInstanceUID', 'PatientAge', 'PatientSex']]

# Convert age to numeric
df_meta['PatientAge'] = df_meta['PatientAge'].astype('float32')

# Encode PatientSex (e.g. M → 1, F → 0)
le = LabelEncoder()
df_meta['PatientSex'] = le.fit_transform(df_meta['PatientSex'].astype(str))

# Merge with your other dataframe if needed
df_merged = df_flayer.merge(df_meta, on='SeriesInstanceUID', how='left')

print(df_meta['PatientSex'].value_counts())
print(df_meta.head())

# Save the LabelEncoder
with open('label_encoder_sex.pkl', 'wb') as f:
    pickle.dump(le, f)

PatientSex
0    3005
1    1343
Name: count, dtype: int64
                                   SeriesInstanceUID  PatientAge  PatientSex
0  1.2.826.0.1.3680043.8.498.10004044428023505108...        64.0           0
1  1.2.826.0.1.3680043.8.498.10004684224894397679...        76.0           0
2  1.2.826.0.1.3680043.8.498.10005158603912009425...        58.0           1
3  1.2.826.0.1.3680043.8.498.10009383108068795488...        71.0           1
4  1.2.826.0.1.3680043.8.498.10012790035410518400...        48.0           0


In [8]:
all_folds_yolo11m = {
    "fold0": {
        "y11_yolo11m": "/kaggle/input/rsna-yolo-models/yolo-11m-2.5D_fold0",
    },
    "fold1": {
       "y11_yolo11m": "/kaggle/input/rsna-yolo-models/yolo-11m-2.5D_fold1",
    },
    "fold2": {
        "y11_yolo11m": "/kaggle/input/rsna-yolo-models/yolo-11m-2.5D_fold22",
    },
    "fold3": {
        "y11_yolo11m": "/kaggle/input/rsna-yolo-models/yolo-11m-2.5D_fold3",
    },
    "fold4": {
        "y11_yolo11m": "/kaggle/input/rsna-yolo-models/yolo-11m-2.5D_fold4",
    },
}


all_folds_yolo_eff2s = {
    "fold0": {
        "y11_effnetv_25d": "/kaggle/input/rsna-yolo-models/cv_effnetv2_s_drop_path_25d_fold0",
    },
    "fold1": {
        "y11_effnetv_25d": "/kaggle/input/rsna-yolo-models/cv_effnetv2_s_drop_path_25d_fold1",
    },
    "fold2": {
        "y11_effnetv_25d": "/kaggle/input/rsna-yolo-models/cv_effnetv2_s_drop_path_25d_fold2",
    },
    "fold3": {
        "y11_effnetv_25d": "/kaggle/input/rsna-yolo-models/cv_effnetv2_s_drop_path_25d_fold3",
    },
    "fold4": {
        "y11_effnetv_25d": "/kaggle/input/rsna-yolo-models/cv_effnetv2_s_drop_path_25d_fold4",
    },
}


def get_yolo_oof_preds(all_folds):
    results = {}
    for fold, models in all_folds.items():
        fold_results = {}
        for model_name, base in models.items():
            matches = glob.glob(f"{base}/**/per_series_predictions.csv", recursive=True)
            fold_results[model_name] = matches[0] if matches else None
        results[fold] = fold_results
    
    dfs = []
    for fold, models in results.items():
        for model_name, csv_path in models.items():
            if csv_path:
                df = pd.read_csv(csv_path)
                df["fold_id"] = fold
                df["model"] = model_name
                dfs.append(df)
    
    if dfs:
        all_df = pd.concat(dfs, ignore_index=True)
        all_df.to_csv("all_folds_series_predictions.csv", index=False)
        print(f"✅ Combined {len(dfs)} CSV files into all_folds_series_predictions.csv")
    else:
        print("⚠️ No CSV files found.")
    return all_df

In [9]:
all_df_yolo11m = get_yolo_oof_preds(all_folds_yolo11m)
all_df_yolo11m.rename(columns={loc_col: f'yolo11m_{loc_col}' for loc_col in loc_cols}, inplace=True)
all_df_yolo11m.rename(columns={'aneurysm_prob': 'yolo11m_aneurysm_prob'}, inplace=True)

all_df_yolo_eff2s = get_yolo_oof_preds(all_folds_yolo_eff2s)
all_df_yolo_eff2s.rename(columns={loc_col: f'yolo_eff2s_{loc_col}' for loc_col in loc_cols}, inplace=True)
all_df_yolo_eff2s.rename(columns={'aneurysm_prob': 'yolo_eff2s_aneurysm_prob'}, inplace=True)

label_to_yolo_idx = {label: YOLO_LABELS_TO_IDX[label] for label in LABEL_COLS}


yolo11m_loc_cols = [f'yolo11m_{loc_col}' for loc_col in loc_cols]
yolo_eff2s_loc_cols = [f'yolo_eff2s_{loc_col}' for loc_col in loc_cols]

yolo11m_loc_col_map = [f"yolo11m_loc_prob_{label_to_yolo_idx[label]}" for label in LABEL_COLS]
yolo_eff2s_loc_col_map = [f"yolo_eff2s_loc_prob_{label_to_yolo_idx[label]}" for label in LABEL_COLS]

feature_cols = ['yolo11m_aneurysm_prob'] + yolo11m_loc_cols + ['yolo_eff2s_aneurysm_prob'] + yolo_eff2s_loc_cols + LABEL_COLS +  ['Aneurysm Present'] + ['PatientAge', 'PatientSex']

print(feature_cols)

✅ Combined 5 CSV files into all_folds_series_predictions.csv
✅ Combined 5 CSV files into all_folds_series_predictions.csv
['yolo11m_aneurysm_prob', 'yolo11m_loc_prob_0', 'yolo11m_loc_prob_1', 'yolo11m_loc_prob_2', 'yolo11m_loc_prob_3', 'yolo11m_loc_prob_4', 'yolo11m_loc_prob_5', 'yolo11m_loc_prob_6', 'yolo11m_loc_prob_7', 'yolo11m_loc_prob_8', 'yolo11m_loc_prob_9', 'yolo11m_loc_prob_10', 'yolo11m_loc_prob_11', 'yolo11m_loc_prob_12', 'yolo_eff2s_aneurysm_prob', 'yolo_eff2s_loc_prob_0', 'yolo_eff2s_loc_prob_1', 'yolo_eff2s_loc_prob_2', 'yolo_eff2s_loc_prob_3', 'yolo_eff2s_loc_prob_4', 'yolo_eff2s_loc_prob_5', 'yolo_eff2s_loc_prob_6', 'yolo_eff2s_loc_prob_7', 'yolo_eff2s_loc_prob_8', 'yolo_eff2s_loc_prob_9', 'yolo_eff2s_loc_prob_10', 'yolo_eff2s_loc_prob_11', 'yolo_eff2s_loc_prob_12', 'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery', 'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery', 'Left Middle Cerebral Art

In [29]:
merged = pd.merge(all_df_yolo11m, all_df_yolo_eff2s, on="SeriesInstanceUID", how="inner")
merged = pd.merge(merged, df_flayer, on="SeriesInstanceUID", how="inner")
merged = pd.merge(merged, df_meta, on="SeriesInstanceUID", how="inner")
merged['fold_id'] = all_df_yolo11m['fold_id'].copy()
df_main = df_main.set_index('SeriesInstanceUID').loc[merged['SeriesInstanceUID']].reset_index()

In [30]:
def train_meta_multilabel_lgb(df):
    """
    Multi-label LightGBM training: trains one binary classifier per label.
    Computes OOF predictions and per-label AUROC.
    """
    features = feature_cols
    labels = LABEL_COLS + ["Aneurysm Present"]
    X = df[features]
    folds = df["fold_id"]

    oof_preds = pd.DataFrame(index=df.index, columns=labels, dtype=float)
    feature_importances = pd.DataFrame()
    feature_importances["feature"] = features

    unique_folds = folds.unique()
    best_iterations = []

    for label in labels:
        print(f"\n=== Training for label: {label} ===")
        oof_preds_label = np.zeros(len(df))
        for fold in unique_folds:
            print(f"\n--- Fold {fold} ---")
            train_idx = df[df["fold_id"] != fold].index
            valid_idx = df[df["fold_id"] == fold].index

            X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
            y_train, y_valid = df_main.loc[train_idx, label], df_main.loc[valid_idx, label]

            model = lgb.LGBMClassifier(
                objective="binary",
                metric="auc",
                max_depth=2,
                learning_rate=0.02,
                reg_lambda=0.0047,
                n_estimators=2000,
                random_state=0,
                verbose=-1
            )

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric="auc",
                callbacks=[early_stopping(stopping_rounds=300, verbose=True)]
            )

            oof_preds_label[valid_idx] = model.predict_proba(X_valid)[:, 1]
            feature_importances[f"{label}_fold_{fold}"] = model.feature_importances_
            joblib.dump(model, f"lgb/meta_classifier_{label}_fold_{fold}.pkl")
            best_iterations.append(model.best_iteration_)

        oof_preds[label] = oof_preds_label
        label_auroc = roc_auc_score(df_main[label[:]], oof_preds_label)
        print(f"=== OOF AUROC for {label}: {label_auroc:.4f} ===")
    cls_score = weighted_multilabel_auc(df_main["Aneurysm Present"].values[:, None], oof_preds["Aneurysm Present"].values[:, None])
    loc_score = weighted_multilabel_auc(df_main[LABEL_COLS].values, oof_preds[LABEL_COLS].values)
    score = 0.5 * cls_score + 0.5 * loc_score
    print(f'kaggle score: {score}')

    # Average feature importance across all folds/labels
    fi_cols = [col for col in feature_importances.columns if col != "feature"]
    feature_importances["average"] = feature_importances[fi_cols].mean(axis=1)
    feature_importances = feature_importances.sort_values(by="average", ascending=False)

    #print("\nFeature Importances:")
    #print(feature_importances)

    return oof_preds, feature_importances

In [31]:
oof_preds_lgb, feature_importances_lgb =  train_meta_multilabel_lgb(merged)


=== Training for label: Left Infraclinoid Internal Carotid Artery ===

--- Fold fold0 ---
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[34]	valid_0's auc: 0.750037

--- Fold fold1 ---
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[224]	valid_0's auc: 0.877466

--- Fold fold2 ---
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[204]	valid_0's auc: 0.815944

--- Fold fold3 ---
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[71]	valid_0's auc: 0.778762

--- Fold fold4 ---
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[114]	valid_0's auc: 0.832123
=== OOF AUROC for Left Infraclinoid Internal Carotid Artery: 0.8023 ===

=== Training for label: Right Infraclinoid Internal Carotid Artery ===

--- Fold fold0 ---
Training until validation scores

In [32]:
def train_meta_multilabel_xgb(df):
    """
    Multi-label XGBoost training: trains one binary classifier per label.
    Computes OOF predictions and per-label AUROC.
    """
    features = feature_cols
    labels = LABEL_COLS + ["Aneurysm Present"]
    X = df[features]
    folds = df["fold_id"]

    oof_preds = pd.DataFrame(index=df.index, columns=labels, dtype=float)
    feature_importances = pd.DataFrame()
    feature_importances["feature"] = features

    unique_folds = folds.unique()
    best_iterations = []

    for label in labels:
        print(f"\n=== Training for label: {label} ===")
        oof_preds_label = np.zeros(len(df))

        for fold in unique_folds:
            print(f"\n--- Fold {fold} ---")
            train_idx = df[df["fold_id"] != fold].index
            valid_idx = df[df["fold_id"] == fold].index

            X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
            y_train, y_valid = df_main.loc[train_idx, label], df_main.loc[valid_idx, label]

            model = xgb.XGBClassifier(
                objective="binary:logistic",
                eval_metric="auc",
                max_depth=2,
                learning_rate=0.02,
                reg_lambda=0.0047,
                n_estimators=2000,
                random_state=0,
                #tree_method="hist",       # faster for large datasets
                n_jobs=-1,
                verbosity=0
            )

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                callbacks=[EarlyStopping(rounds=300, save_best=True)],
                verbose = False
            )

            # predict using the best iteration
            oof_preds_label[valid_idx] = model.predict_proba(X_valid, iteration_range=(0, model.best_iteration + 1))[:, 1]

            # Save feature importance for each fold
            feature_importances[f"{label}_fold_{fold}"] = model.feature_importances_

            # Save model
            joblib.dump(model, f"xgb/meta_classifier_{label}_fold_{fold}.pkl")
            best_iterations.append(model.best_iteration)

        oof_preds[label] = oof_preds_label
        label_auroc = roc_auc_score(df_main[label], oof_preds_label)
        print(f"=== OOF AUROC for {label}: {label_auroc:.4f} ===")

    # Compute Kaggle-like score
    cls_score = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        oof_preds["Aneurysm Present"].values[:, None]
    )
    loc_score = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        oof_preds[LABEL_COLS].values
    )
    score = 0.5 * cls_score + 0.5 * loc_score
    print(f'kaggle score: {score:.6f}')

    # Average feature importance across all folds/labels
    fi_cols = [col for col in feature_importances.columns if col != "feature"]
    feature_importances["average"] = feature_importances[fi_cols].mean(axis=1)
    feature_importances = feature_importances.sort_values(by="average", ascending=False)

    return oof_preds, feature_importances

In [33]:
oof_preds_xgb, feature_importances_xgb =  train_meta_multilabel_xgb(merged)


=== Training for label: Left Infraclinoid Internal Carotid Artery ===

--- Fold fold0 ---


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(



--- Fold fold1 ---


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(



--- Fold fold2 ---


...


--- Fold fold3 ---


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(



--- Fold fold4 ---


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


=== OOF AUROC for Aneurysm Present: 0.8527 ===
kaggle score: 0.853271


In [34]:
def train_meta_multilabel_catboost(df):
    """
    Multi-label CatBoost training: trains one binary classifier per label.
    Computes OOF predictions and per-label AUROC.
    """
    features = feature_cols
    labels = LABEL_COLS + ["Aneurysm Present"]
    X = df[features]
    folds = df["fold_id"]

    oof_preds = pd.DataFrame(index=df.index, columns=labels, dtype=float)
    feature_importances = pd.DataFrame()
    feature_importances["feature"] = features

    unique_folds = folds.unique()
    best_iterations = []

    # Shared CatBoost parameters
    params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'depth': 6,
        'learning_rate': 0.02,
        'l2_leaf_reg': 4.7,
        'iterations': 2000,
        'random_seed': 0,
        'bootstrap_type': 'Bayesian',
        'bagging_temperature': 1.0,
        'od_type': 'Iter',
        'od_wait': 300,
        'verbose': False
    }

    for label in labels:
        print(f"\n=== Training for label: {label} ===")
        oof_preds_label = np.zeros(len(df))

        for fold in unique_folds:
            print(f"\n--- Fold {fold} ---")
            train_idx = df[df["fold_id"] != fold].index
            valid_idx = df[df["fold_id"] == fold].index

            X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
            y_train, y_valid = df_main.loc[train_idx, label], df_main.loc[valid_idx, label]

            train_pool = Pool(X_train, y_train)
            valid_pool = Pool(X_valid, y_valid)

            model = CatBoostClassifier(**params)

            model.fit(
                train_pool,
                eval_set=valid_pool,
                use_best_model=True
            )

            oof_preds_label[valid_idx] = model.predict_proba(X_valid)[:, 1]
            feature_importances[f"{label}_fold_{fold}"] = model.get_feature_importance(train_pool)

            joblib.dump(model, f"cat/meta_classifier_{label}_fold_{fold}.pkl")
            best_iterations.append(model.get_best_iteration())

        oof_preds[label] = oof_preds_label
        label_auroc = roc_auc_score(df_main[label], oof_preds_label)
        print(f"=== OOF AUROC for {label}: {label_auroc:.4f} ===")

    # Combine aneurysm/global score
    cls_score = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        oof_preds["Aneurysm Present"].values[:, None]
    )
    loc_score = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        oof_preds[LABEL_COLS].values
    )
    score = 0.5 * cls_score + 0.5 * loc_score
    print(f'kaggle score: {score:.6f}')

    # Average feature importance across all folds and labels
    fi_cols = [col for col in feature_importances.columns if col != "feature"]
    feature_importances["average"] = feature_importances[fi_cols].mean(axis=1)
    feature_importances = feature_importances.sort_values(by="average", ascending=False)

    return oof_preds, feature_importances

In [35]:
oof_preds_cat, feature_importances_cat =  train_meta_multilabel_catboost(merged)


=== Training for label: Left Infraclinoid Internal Carotid Artery ===

--- Fold fold0 ---

--- Fold fold1 ---

--- Fold fold2 ---

--- Fold fold3 ---

--- Fold fold4 ---
=== OOF AUROC for Left Infraclinoid Internal Carotid Artery: 0.7832 ===

=== Training for label: Right Infraclinoid Internal Carotid Artery ===

--- Fold fold0 ---

--- Fold fold1 ---

--- Fold fold2 ---

--- Fold fold3 ---

--- Fold fold4 ---
=== OOF AUROC for Right Infraclinoid Internal Carotid Artery: 0.7638 ===

=== Training for label: Left Supraclinoid Internal Carotid Artery ===

--- Fold fold0 ---

--- Fold fold1 ---

--- Fold fold2 ---

--- Fold fold3 ---

--- Fold fold4 ---
=== OOF AUROC for Left Supraclinoid Internal Carotid Artery: 0.8478 ===

=== Training for label: Right Supraclinoid Internal Carotid Artery ===

--- Fold fold0 ---

--- Fold fold1 ---

--- Fold fold2 ---

--- Fold fold3 ---

--- Fold fold4 ---
=== OOF AUROC for Right Supraclinoid Internal Carotid Artery: 0.7528 ===

=== Training for label:

In [36]:
def compute_score(p_cls, p_loc):
    return 0.5 * (p_cls + p_loc)

gt_loc = df_main[LABEL_COLS]
pred_loc = oof_preds_lgb[LABEL_COLS] + oof_preds_xgb[LABEL_COLS] + oof_preds_cat[LABEL_COLS] + merged[LABEL_COLS] + merged[yolo11m_loc_col_map].values + merged[yolo_eff2s_loc_col_map].values
score_loc = weighted_multilabel_auc(gt_loc.values, pred_loc.values)

gt_cls = df_main['Aneurysm Present']
pred_cls = oof_preds_lgb['Aneurysm Present'] + oof_preds_xgb['Aneurysm Present'] + oof_preds_cat['Aneurysm Present'] + merged['Aneurysm Present'] + merged['yolo11m_aneurysm_prob']  + merged['yolo_eff2s_aneurysm_prob']
score_cls = weighted_multilabel_auc(gt_cls.values[:, None], pred_cls.values[:, None])
compute_score(score_cls, score_loc)

0.852141393707105

In [37]:
# Prepare prediction arrays
preds_loc_list = [
    oof_preds_lgb[LABEL_COLS].values,
    oof_preds_xgb[LABEL_COLS].values,
    oof_preds_cat[LABEL_COLS].values,
    merged[LABEL_COLS].values,
    merged[yolo11m_loc_col_map].values,
    merged[yolo_eff2s_loc_col_map].values
]

preds_cls_list = [
    oof_preds_lgb['Aneurysm Present'].values,
    oof_preds_xgb['Aneurysm Present'].values,
    oof_preds_cat['Aneurysm Present'].values,
    merged['Aneurysm Present'].values,
    merged['yolo11m_aneurysm_prob'].values,
    merged['yolo_eff2s_aneurysm_prob'].values
]

gt_loc_values = df_main[LABEL_COLS].values
gt_cls_values = df_main['Aneurysm Present'].values[:, None]


# Objective function to **minimize** (negative score)
def objective(weights):
    # Normalize weights to sum 1 (optional)
    weights = np.array(weights)
    weights /= weights.sum()

    # Weighted ensemble
    pred_loc_weighted = sum(w * p for w, p in zip(weights, preds_loc_list))
    pred_cls_weighted = sum(w * p for w, p in zip(weights, preds_cls_list))

    # Compute individual scores
    score_loc = weighted_multilabel_auc(gt_loc_values, pred_loc_weighted)
    score_cls = weighted_multilabel_auc(gt_cls_values, pred_cls_weighted)

    # Compute combined score
    combined_score = compute_score(score_cls, score_loc)

    # Return negative because we minimize
    return -combined_score

# Initial weights
init_weights = [1/len(preds_loc_list)] * len(preds_loc_list)

# Run Nelder-Mead optimization
result = minimize(objective, init_weights, method='Nelder-Mead', options={'maxiter': 1000, 'disp': True})

# Optimized weights
optimized_weights = result.x / result.x.sum()
print("Optimized weights:", optimized_weights)
print("Maximized score:", -result.fun)

Optimization terminated successfully.
         Current function value: -0.858995
         Iterations: 139
         Function evaluations: 246
Optimized weights: [0.2793572  0.58535173 0.00420708 0.01056496 0.06308367 0.05743536]
Maximized score: 0.8589951003546813


In [38]:
def compute_auc_table():
    results = []

    # LGB
    cls_auc_lgb = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        oof_preds_lgb["Aneurysm Present"].values[:, None]
    )
    loc_auc_lgb = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        oof_preds_lgb[LABEL_COLS].values
    )
    results.append(["LightGBM", loc_auc_lgb, cls_auc_lgb, 0.5 * (loc_auc_lgb + cls_auc_lgb)])

    # XGB
    cls_auc_xgb = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        oof_preds_xgb["Aneurysm Present"].values[:, None]
    )
    loc_auc_xgb = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        oof_preds_xgb[LABEL_COLS].values
    )
    results.append(["XGBoost", loc_auc_xgb, cls_auc_xgb, 0.5 * (loc_auc_xgb + cls_auc_xgb)])

    # CatBoost
    cls_auc_cat = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        oof_preds_cat["Aneurysm Present"].values[:, None]
    )
    loc_auc_cat = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        oof_preds_cat[LABEL_COLS].values
    )
    results.append(["CatBoost", loc_auc_cat, cls_auc_cat, 0.5 * (loc_auc_cat + cls_auc_cat)])

    # YOLO11m
    cls_auc_y11m = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        merged["yolo11m_aneurysm_prob"].values[:, None]
    )
    loc_auc_y11m = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        merged[yolo11m_loc_col_map].values
    )
    results.append(["YOLO11m", loc_auc_y11m, cls_auc_y11m, 0.5 * (loc_auc_y11m + cls_auc_y11m)])

    # YOLO EfficientNetV2-S
    cls_auc_eff2s = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        merged["yolo_eff2s_aneurysm_prob"].values[:, None]
    )
    loc_auc_eff2s = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        merged[yolo_eff2s_loc_col_map].values
    )
    results.append(["YOLO EffNetV2-S", loc_auc_eff2s, cls_auc_eff2s, 0.5 * (loc_auc_eff2s + cls_auc_eff2s)])

    # F-layer baseline (if available)
    cls_auc_flayer = weighted_multilabel_auc(
        df_main["Aneurysm Present"].values[:, None],
        merged["Aneurysm Present"].values[:, None]
    )
    loc_auc_flayer = weighted_multilabel_auc(
        df_main[LABEL_COLS].values,
        merged[LABEL_COLS].values
    )
    results.append(["F-layer Baseline", loc_auc_flayer, cls_auc_flayer, 0.5 * (loc_auc_flayer + cls_auc_flayer)])

    # Combine results
    table = pd.DataFrame(results, columns=["Model", "Loc AUC", "Cls AUC", "Kaggle Score"])
    print("\n=== AUC Summary Table ===")
    print(table.to_markdown(index=False))
    return table

# Call the summary function
auc_table = compute_auc_table()


=== AUC Summary Table ===
| Model            |   Loc AUC |   Cls AUC |   Kaggle Score |
|:-----------------|----------:|----------:|---------------:|
| LightGBM         |  0.862306 |  0.852513 |       0.857409 |
| XGBoost          |  0.85389  |  0.852653 |       0.853271 |
| CatBoost         |  0.82411  |  0.854372 |       0.839241 |
| YOLO11m          |  0.822321 |  0.786436 |       0.804378 |
| YOLO EffNetV2-S  |  0.818497 |  0.807531 |       0.813014 |
| F-layer Baseline |  0.755192 |  0.790901 |       0.773047 |
